In [6]:
import collections

from calculators.total import TotalCostCalculator
import numpy as np

In [5]:
def linrange(start, stop, n=50):
	return np.ceil(np.linspace(start, stop, n)).astype(int)


def user_month_rate(calculator: TotalCostCalculator):
	cost = calculator.calculate_cost()
	return cost / calculator.node.users

In [8]:
defaults = {
	'users': 1e3,
	'msg_size_in_kb': 16,
	'msg_processing_in_secs': 1,
	'polling_in_secs': 10,
	'batch_size': 100,
	'batch_window_in_secs': 10,
	'msgs_received_per_min': 30,
	'concurrency': 10,
	'function_memory_in_mb': 256,
	'use_s3': True,
	'monthly_code_uploads': 1,
	'monthly_node_creations': 50,
	'monthly_node_deletions': 25,
	'existing_nodes': 1e3,
	'code_upload_in_mb': 256,
	'existing_code_in_mb': 0,
	'code_metadata_in_kb': 8,
	'node_metadata_in_kb': 8,
}

TotalCostCalculator(**defaults).calculate_cost()

104.93831459864

In [9]:

defaults_increased_polling = {**defaults, 'polling_in_secs': 15}
TotalCostCalculator(**defaults_increased_polling).calculate_cost()

52.37831459864001

In [ ]:
ranges = {
	'users': linrange(1, 1e5),
	'msg_size_in_kb': linrange(1, 256),
	'msg_processing_in_secs': linrange(0.1, 10),
	'polling_in_secs': linrange(1, 20),
	'batch_size': linrange(1, 1e3),
	'batch_window_in_secs': linrange(1, 30),
	'msgs_received_per_min': linrange(1, 120),
	'concurrency': linrange(5, 50),
	'function_memory_in_mb': linrange(128, 1024),
	'use_s3': (True, False),
	'monthly_code_uploads': linrange(1, 1e2),
	'monthly_node_creations': linrange(1, 1e2),
	'monthly_node_deletions': linrange(1, 1e2),
	'existing_nodes': linrange(0.1, 1e3),
	'code_upload_in_mb': linrange(0.1, 1e3),
	'existing_code_in_mb': linrange(0.1, 1e4),
	'code_metadata_in_kb': linrange(0.1, 1e3),
	'node_metadata_in_kb': linrange(0.1, 1e3),
}

In [169]:
results = collections.defaultdict(list)

for param, values in ranges.items():
	kwargs = {**defaults}
	for value in values:
		kwargs[param] = value
		calc = TotalCostCalculator(**kwargs)
		results[param].append(user_month_rate(calc))
	results[param] = np.array(results[param])

stats = {
	param: (np.mean(values), np.var(values))
	for param, values in results.items()
}

stats

{'users': (0.1033759184427382, 0.00014766199093914592),
 'msg_size_in_kb': (0.18377831459864002, 0.0035636941440000025),
 'msg_processing_in_secs': (0.10493831459864002, 1.925929944387236e-34),
 'polling_in_secs': (0.0969847606780714, 0.009034126123865737),
 'batch_size': (0.10493831459864002, 1.925929944387236e-34),
 'batch_window_in_secs': (0.06597247859864, 0.00021877076863630399),
 'msgs_received_per_min': (0.15911015459864003, 0.003771418529894401),
 'concurrency': (0.10493831459864002, 1.925929944387236e-34),
 'function_memory_in_mb': (0.10493831459864002, 1.925929944387236e-34),
 'use_s3': (0.10494792709864001, 9.240015624997804e-11),
 'monthly_code_uploads': (0.10572579947864, 2.0728496822866531e-07),
 'monthly_node_creations': (0.10493832650563999, 1.2326177600131439e-13),
 'monthly_node_deletions': (0.10493836006364, 2.5571250250874767e-15),
 'existing_nodes': (0.10493236735428829, 1.229276784050423e-11),
 'code_upload_in_mb': (0.10494380719629623, 4.373964954720137e-11),
 'e

In [170]:
def sort_by_var():
	ranked = sorted(stats.items(), key=lambda x: x[1][1], reverse=True)
	return {k: v for k, v in ranked}

In [173]:
var_ranked = sort_by_var()
variances = {k: v[1] for k, v in var_ranked.items()}
variances

{'polling_in_secs': 0.009034126123865737,
 'msgs_received_per_min': 0.003771418529894401,
 'msg_size_in_kb': 0.0035636941440000025,
 'batch_window_in_secs': 0.00021877076863630399,
 'users': 0.00014766199093914592,
 'monthly_code_uploads': 2.0728496822866531e-07,
 'code_metadata_in_kb': 1.3546888812499996e-07,
 'node_metadata_in_kb': 8.222037962248307e-09,
 'existing_code_in_mb': 4.375424600528407e-09,
 'use_s3': 9.240015624997804e-11,
 'code_upload_in_mb': 4.373964954720137e-11,
 'existing_nodes': 1.229276784050423e-11,
 'monthly_node_creations': 1.2326177600131439e-13,
 'monthly_node_deletions': 2.5571250250874767e-15,
 'msg_processing_in_secs': 1.925929944387236e-34,
 'batch_size': 1.925929944387236e-34,
 'concurrency': 1.925929944387236e-34,
 'function_memory_in_mb': 1.925929944387236e-34}

In [ ]:
max_var = max(variances.values())
normalized = {k: v / max_var for k, v in variances.items()}

In [172]:
normalized

{'polling_in_secs': 1.0,
 'msgs_received_per_min': 0.41746356849405997,
 'msg_size_in_kb': 0.3944702669786377,
 'batch_window_in_secs': 0.02421604100239096,
 'users': 0.01634491138540368,
 'monthly_code_uploads': 2.2944661762145878e-05,
 'code_metadata_in_kb': 1.4995239856916267e-05,
 'node_metadata_in_kb': 9.101088306181478e-07,
 'existing_code_in_mb': 4.843218414861079e-07,
 'use_s3': 1.0227901955661391e-08,
 'code_upload_in_mb': 4.8416027125914206e-09,
 'existing_nodes': 1.3607035890311555e-09,
 'monthly_node_creations': 1.3644017618448989e-11,
 'monthly_node_deletions': 2.830517296335103e-13,
 'msg_processing_in_secs': 2.1318386725855484e-32,
 'batch_size': 2.1318386725855484e-32,
 'concurrency': 2.1318386725855484e-32,
 'function_memory_in_mb': 2.1318386725855484e-32}